Note: a continuation from Histograms_sentences_scores_feature_selection.ipynb

We calculate feature importance off of the test set's features because only the test set's sentences get scored (probssentences). Is this legal?

In [2]:
import pickle
import datetime
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [3]:
probssentences = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/probssentences.p', 'rb'))
test1 = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/test1.p', 'rb'))

In [4]:
features = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/features.p', 'rb'))
labels = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/labels.p', 'rb'))

In [5]:
#get feature weights for each sentence score from 0 to 1 in bins of 10
clf = ExtraTreesClassifier()
clf = clf.fit(features, labels)
weights = clf.feature_importances_  
#Normalize so all are greater than 1
weights = weights * 100
print weights

[ 32.25557907   5.14741849   4.07540472   3.25938003   3.49981633
   3.7477608    3.70348957   4.12769366   5.27599952  34.9074578 ]


In [6]:
#is the issue that the ET classifier used is fit on the test data? I only want the weights, not the model itself.

In [6]:
#Apply to the actual sentence scores
a = datetime.datetime.now().replace(microsecond=0)

probssentences['0_weighted'] = 9.99
probssentences['1_weighted'] = 9.99

for i in range(len(probssentences)):
    valneg = probssentences.iloc[i][0]
    inxneg = 99999
    if 0 <= valneg < 0.1:
        inxneg = 0
    elif 0.1 <= valneg < 0.2:
        inxneg = 1
    elif 0.2 <= valneg < 0.3:
        inxneg = 2
    elif 0.3 <= valneg < 0.4:
        inxneg = 3
    elif 0.4 <= valneg < 0.5:
        inxneg = 4
    elif 0.5 <= valneg < 0.6:
        inxneg = 5
    elif 0.6 <= valneg < 0.7:
        inxneg = 6
    elif 0.7 <= valneg < 0.8:
        inxneg = 7
    elif 0.8 <= valneg < 0.9:
        inxneg = 8
    elif 0.9 <= valneg <= 1.0:
        inxneg = 9
    probssentences.set_value(i, '0_weighted', valneg * weights[inxneg])
    
    valpos = probssentences.iloc[i][1]
    inxpos = 9.9
    if 0 <= valpos < 0.1:
        inxpos = 0
    elif 0.1 <= valpos < 0.2:
        inxpos = 1
    elif 0.2 <= valpos < 0.3:
        inxpos = 2
    elif 0.3 <= valpos < 0.4:
        inxpos = 3
    elif 0.4 <= valpos < 0.5:
        inxpos = 4
    elif 0.5 <= valpos < 0.6:
        inxpos = 5
    elif 0.6 <= valpos < 0.7:
        inxpos = 6
    elif 0.7 <= valpos < 0.8:
        inxpos = 7
    elif 0.8 <= valpos < 0.9:
        inxpos = 8
    elif 0.9 <= valpos <= 1.0:
        inxpos = 9
    probssentences.set_value(i, '1_weighted', valpos * weights[inxpos])
    
b = datetime.datetime.now().replace(microsecond=0)
print b-a

0:00:44


In [33]:
print probssentences[0:20]

             0            1 doc  0_weighted  1_weighted
0       0.5986       0.4014   0    2.243410    1.404826
1   0.00792023      0.99208   0    0.255472   34.630984
2     0.369248     0.630752   0    1.203518    2.335985
3     0.445318     0.554682   0    1.558533    2.078814
4     0.999581  0.000419095   0   34.892829    0.013518
5   0.00449226     0.995508   1    0.144900   34.750644
6     0.918007    0.0819928   1   32.045295    2.644727
7      0.55223      0.44777   1    2.069627    1.567112
8     0.986808    0.0131918   1   34.446967    0.425509
9     0.721142     0.278858   1    2.976654    1.136459
10    0.111639     0.888361   1    0.574651    4.686995
11    0.717194     0.282805   1    2.960359    1.152547
12   0.0193013     0.980699   2    0.622574   34.233697
13   0.0159692     0.984031   2    0.515094   34.350015
14   0.0224181     0.977582   2    0.723108   34.124900
15    0.986999    0.0130013   2   34.453617    0.419364
16   0.0593864     0.940614   3    1.915544   32

In [8]:
prob = probssentences.groupby("doc").mean()
print prob[0:10]

     0_weighted  1_weighted
doc                        
0      8.030752    8.092825
1     10.745493    6.623427
2      9.078598   25.781994
3      1.555897   33.223643
4      9.947820    7.642471
5      1.431370   10.375656
6     13.528790   12.809105
7      0.207365   34.683045
8     10.848690    6.896297
9     34.639876    0.247252


In [9]:
preds = np.ones((prob.shape[0]))
print preds.shape

(12500,)


In [10]:
preds[np.where(prob.iloc[:,0] >  prob.iloc[:,1])] = 0 # The first column is the negative model

In [11]:
print preds.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [12]:
print np.size(np.where(preds == test1["sentiment"]))*1./np.size(preds)

0.86848


In [23]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(test1["sentiment"], preds)

0.868160030313


Especially considering the weights are fit from the test data itself, we would expect to see a big increase in performance, which we're not getting.

Why? :/ could have something to do with the fact that we fit a classifier on the test data which is pretty wacky. Or could be the way the weights were applied to the sentence scores (unlikely). 

### This performance is SO similar to word2vec, check if these are the same indices or not.
Nope! 463 different of 12500.

In [14]:
w2vpreds = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/predictions.p', 'rb'))

In [22]:
print roc_auc_score(test1["sentiment"], w2vpreds)

0.866634984213


In [16]:
print type(preds)
print type(w2vpreds)

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [18]:
np.array_equal(preds, w2vpreds)

False

In [21]:
print len(np.where(preds!=w2vpreds)[0])

463


In [27]:
#print indices where w2v was wrong and new method was right
print len(np.where((w2vpreds != test1['sentiment']) & (preds != w2vpreds))[0])
print len(np.where((preds != test1['sentiment']) & (preds != w2vpreds))[0])

241
222


In [28]:
np.where((w2vpreds != test1['sentiment']) & (preds != w2vpreds))[0]

array([   60,   131,   156,   243,   246,   250,   280,   366,   380,
         478,   485,   533,   565,   614,   647,   667,   755,   820,
         867,   879,   991,  1010,  1023,  1078,  1079,  1170,  1221,
        1294,  1296,  1323,  1348,  1461,  1471,  1645,  1706,  1759,
        1804,  1913,  1922,  2050,  2063,  2090,  2140,  2162,  2297,
        2305,  2335,  2480,  2631,  2634,  2660,  2682,  2710,  2743,
        2748,  2773,  2894,  2936,  2973,  2989,  3016,  3021,  3022,
        3038,  3039,  3060,  3102,  3154,  3178,  3210,  3262,  3286,
        3325,  3332,  3350,  3365,  3533,  3657,  3861,  3864,  3879,
        3914,  3956,  4056,  4113,  4242,  4250,  4412,  4552,  4563,
        4627,  4682,  4748,  4773,  4800,  4801,  4870,  4920,  4986,
        5005,  5012,  5204,  5222,  5249,  5269,  5272,  5321,  5450,
        5471,  5636,  5648,  5790,  5877,  5935,  5937,  5950,  5995,
        6014,  6015,  6074,  6240,  6308,  6467,  6486,  6502,  6524,
        6578,  6611,

In [32]:
print probssentences.loc[probssentences['doc']==60]
#.groupby('doc').mean()

               0            1 doc  0_weighted  1_weighted
695     0.986983    0.0130174  60   34.453053    0.419882
696    0.0317208     0.968279  60    1.023172   33.800167
697     0.653628     0.346372  60    2.420704    1.128958
698     0.997484   0.00251651  60   34.819615    0.081172
699    0.0193894     0.980611  60    0.625418   34.230623
700     0.999291  0.000708568  60   34.882723    0.022855
701     0.126453     0.873547  60    0.650905    4.608835
702     0.579535     0.420465  60    2.171959    1.471550
703  0.000281299     0.999719  60    0.009073   34.897639


In [46]:
print probssentences.loc[probssentences['doc']==60].groupby('doc')